In [38]:
'''
匯入套件
'''

# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#美麗湯
from bs4 import BeautifulSoup as bs

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

#正規表達式
import re

# 下載檔案的工具
import wget

import pandas as pd


# 由於klook 會擋 嘗試更換user-agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument(f"user-agent={user_agent}")

 # 加入其他可能有用的隱私設定
my_options.add_argument('--disable-blink-features=AutomationControlled')
my_options.add_argument('--disable-extensions')
my_options.add_experimental_option('excludeSwitches', ['enable-automation'])
my_options.add_experimental_option('useAutomationExtension', False)
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 放置爬取的資料
listData = []

In [39]:
# 前往klook 頁面
url = 'https://www.klook.com/zh-TW/activity/10136-national-palace-museum-ticket-package-taipei/'
driver.get(url)

# 清空列表
listData.clear()

# 設定頁面滾動初始值
innerHeight = 0
offset = 0
count = 0
limit = 3
    
# 在捲動到沒有元素動態產生前，持續捲動
while count <= limit:
    # 每次移動高度
    offset = driver.execute_script(
        'return document.documentElement.scrollHeight;'
    )

    # 捲軸往下滑動
    driver.execute_script(f'''
        window.scrollTo({{
            top: {offset}, 
            behavior: 'smooth' 
        }});
    ''')
        
    # (重要)強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
    sleep(3)
        
    # 透過執行 js 語法來取得捲動後的當前總高度
    innerHeight = driver.execute_script(
        'return document.documentElement.scrollHeight;'
    )
        
    # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
    if offset == innerHeight:
        count += 1

sleep(3)

# 取得網頁原始碼
html = driver.page_source
#print(html)
    
# 用美麗湯重新編譯
soup = bs(html, 'lxml')

# 點擊回復原文按鈕 只有第一次動態生成要按 後面的動態生成頁面都會顯示原文
translate_buttons = driver.find_elements(By.CSS_SELECTOR, "div.comment-item-translate")

for button in translate_buttons:
        # 滾動到按鈕位置
        driver.execute_script("arguments[0].scrollIntoView(true);", button)

        # 顯性等待按鈕可點擊
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.comment-item-translate"))
        )

       # 使用 JavaScript 點擊
        driver.execute_script("arguments[0].click();", button)

        sleep(1)

# 自己設定counter
count = 0

#Klook上限2000筆 一頁八筆 約250頁
while count<250:
    
    # 取得更多內容按鈕
    more_buttons = driver.find_elements(By.CSS_SELECTOR, "span.more-btn")

    for button in more_buttons:
            # 滾動到按鈕位置
        driver.execute_script("arguments[0].scrollIntoView(true);", button)

        try:   
            # 顯性等待按鈕可點擊
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "span.more-btn"))
            )

        # 使用 JavaScript 點擊更多內容按鈕
            driver.execute_script("arguments[0].click();", button)

            sleep(1)
        # 若頁面無此按鈕則跳出迴圈       
        except TimeoutException:
            break

    # 再次取得網頁原始碼
    html = driver.page_source
    #print(html)
        
    # 用美麗湯重新編譯
    soup = bs(html, 'lxml')

    # 取得所有評論
    content = soup.select("p.content")
    # 取得所有評論時間
    date = soup.select("div.review-user-time")
        
    for time,comment in zip(date,content):
            
        # 調整時間格式
        regex = r'\D'
        format_time = re.sub(regex,'',time.get_text().strip())
            
        listData.append({
            'sources':"Klook",
            'title': "review",
            'url':url,
            'time':format_time,
            'results': comment.get_text().strip()
        })

    sleep(2)
    
    #取得下一頁按鈕
    next_button = driver.find_element(By.CSS_SELECTOR, "span.klk-pagination-next-btn")


    # 滾動到按鈕位置
    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)

    try:   
        # 顯性等待按鈕可點擊
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "span.klk-pagination-next-btn"))
        )

        # 使用 JavaScript 點擊下一頁按鈕
        driver.execute_script("arguments[0].click();", next_button)

        sleep(3)

    # 若沒下一頁則跳出迴圈           
    except TimeoutException:
        next_page = False
    
driver.quit()


KeyboardInterrupt: 

In [40]:
# 將結果轉為data frame
df = pd.DataFrame(listData)
# 取得文章數量
print(len(df))
# 輸出為csv檔
df.to_csv('klook', index=True, encoding='utf-8-sig')

13232
